In [1]:
from pyimagesearch.transform import four_point_transform
from pyimagesearch import imutils
from skimage.filters import threshold_adaptive
import numpy as np
import cv2

In [2]:
imgPath = 'C:/Users/Yaroslav/Pictures/For projects/IMG_20170316_115642.jpg'

In [3]:
#Step 1: Edge Detection
# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
image = cv2.imread(imgPath)
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)

In [4]:
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0)
edged = cv2.Canny(gray, 75, 200)

In [5]:
# show the original image and the edge detected image
# cv2.startWindowThread()
# cv2.namedWindow("preview")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
# cv2.startWindowThread()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# STEP 2: Finding Contours
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour

(_,cnts,_) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

screenCnt = 0
#loop over the contours 
for c in cnts:
        lol = 0
        #approximate the contour
        peri = cv2.arcLength(c,True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
            break

In [7]:
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# Step 3: Apply a Perspective Transform & Threshold
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4,2) * ratio)

# convert the wraped image to grayscale, then threshold it
# to give it thet 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
warped = threshold_adaptive(warped, 251, offset = 10)
warped = warped.astype('uint8') * 255

In [9]:
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow('Scanned', imutils.resize(warped, height = 650))
cv2.waitKey(0)

-1